In [11]:
import pennylane as qml
from pennylane.fermi import FermiC, FermiA, jordan_wigner
import networkx as nx
import numpy as np

# Parameters
phi = np.pi / 2 #0.5  # Aharonov-Bohm phase
eta = np.exp(1j * phi)  # Phase factor
N = 4  # Number of sites
edges = [(1, 2), (2, 3), (2, 4), (1, 3), (3, 4)]
red_link = (2, 3) # Edge where Aharonov-Bohm phase applies

# Build connectivity graph
G = nx.Graph()
G.add_edges_from(edges)

# Define path function between qubits
def jw_path(i, j):
    path = nx.shortest_path(G, i, j)
    return path[1:-1] if len(path) > 2 else []

# Get unique i<j with paths
nodes = sorted(set(i for e in edges for i in e))
#triples = [(i, j, jw_path(i, j)) for i in nodes for j in nodes if i < j]
triples = [(min(i,j), max(i,j), jw_path(i, j)) for (i, j) in edges]

# Anyonic Sigma dagger operator (fermionic version)
# def sigma_dagger(k, theta):
#     a_k_dag = FermiC(k)
#     a_k = FermiA(k)
#     n_k = a_k_dag * a_k
#     id_op = qml.fermi.FermiSentence({})
#     z_k = id_op - 2 * n_k
#     #term_I = 0.5 * np.exp(1 + 1j * theta) * id_op
#     #term_Z = 0.5 * np.exp(-1 + 1j * theta) * z_k
#     term_I = 0.5 * np.exp(1j * theta) * 1
#     term_Z = 0.5 * np.exp(1j * theta) * Z
#     return term_I + term_Z

def sigma_dagger(k, theta):
    id_op = qml.fermi.FermiSentence({})
    n_k = FermiC(k) * FermiA(k)
    z_k = id_op - 2 * n_k
    return 0.5 * (np.exp(1j * theta) + 1) * id_op + 0.5 * (np.exp(1j * theta) - 1) * z_k

# Build Hamiltonian using Jordan-Wigner transformation
coeffs = []
ops = []

for (i, j, path) in triples:
    i_idx = i - 1
    j_idx = j - 1
    a_i_dag = FermiC(i_idx)
    a_j = FermiA(j_idx)

    phase = eta if (i, j) == red_link or (j, i) == red_link else 1.0

    # Build Sigma dagger product along path
    sigmas = [sigma_dagger(k - 1, phi) for k in reversed(path)]

    f_op = a_i_dag
    for s in sigmas:
        f_op = f_op * s
    f_op = f_op * a_j
    f_op *= phase

    qubit_op = jordan_wigner(f_op)
    ops.extend([qubit_op, qubit_op.adjoint()])
    coeffs.extend([1.0, 1.0])

# Add interaction terms: n_i n_j
V = 1.0  # Interaction strength
for i in range(N):
    for j in range(i + 1, N):
        ni = FermiC(i) * FermiA(i)
        nj = FermiC(j) * FermiA(j)
        f_interaction = ni * nj
        qubit_op = jordan_wigner(f_interaction)
        ops.append(qubit_op)
        coeffs.append(V)

# Hamiltonian
H = qml.Hamiltonian(coeffs, ops)

# Iterate over each term in the Hamiltonian and print the Pauli string
for i, op in enumerate(H.ops):
    print(f"Pauli string of term {i+1}: {op}")


Pauli string of term 1: -0.25j * (Y(0) @ X(1)) + (0.25+0j) * (Y(0) @ Y(1)) + (0.25+0j) * (X(0) @ X(1)) + 0.25j * (X(0) @ Y(1))
Pauli string of term 2: Adjoint(-0.25j * (Y(0) @ X(1))) + Adjoint((0.25+0j) * (Y(0) @ Y(1))) + Adjoint((0.25+0j) * (X(0) @ X(1))) + Adjoint(0.25j * (X(0) @ Y(1)))
Pauli string of term 3: (0.25-1.5308084989341915e-17j) * (Y(1) @ X(2)) + (1.5308084989341915e-17+0.25j) * (Y(1) @ Y(2)) + (1.5308084989341915e-17+0.25j) * (X(1) @ X(2)) + (-0.25+1.5308084989341915e-17j) * (X(1) @ Y(2))
Pauli string of term 4: Adjoint((0.25-1.5308084989341915e-17j) * (Y(1) @ X(2))) + Adjoint((1.5308084989341915e-17+0.25j) * (Y(1) @ Y(2))) + Adjoint((1.5308084989341915e-17+0.25j) * (X(1) @ X(2))) + Adjoint((-0.25+1.5308084989341915e-17j) * (X(1) @ Y(2)))
Pauli string of term 5: -0.25j * (Y(1) @ Z(2) @ X(3)) + (0.25+0j) * (Y(1) @ Z(2) @ Y(3)) + (0.25+0j) * (X(1) @ Z(2) @ X(3)) + 0.25j * (X(1) @ Z(2) @ Y(3))
Pauli string of term 6: Adjoint(-0.25j * (Y(1) @ Z(2) @ X(3))) + Adjoint((0.25+0j

In [12]:
import pennylane as qml
from pennylane import numpy as np
H_simplified = qml.simplify(H)

In [13]:
print("Pauli decomposition of H:")
for coeff, op in zip(H_simplified.coeffs, H_simplified.ops):
    print(f"{op}: {coeff}")

Pauli decomposition of H:
Y(0) @ Y(1): (0.5+0j)
X(0) @ X(1): (0.5+0j)
Y(1) @ X(2): (0.5+0j)
X(1) @ Y(2): (-0.5+0j)
Y(1) @ Z(2) @ Y(3): (0.5+0j)
X(1) @ Z(2) @ X(3): (0.5+0j)
Y(0) @ Z(1) @ Y(2): (0.5+0j)
X(0) @ Z(1) @ X(2): (0.5+0j)
Y(2) @ Y(3): (0.5+0j)
X(2) @ X(3): (0.5+0j)
I(): (1.5+0j)
Z(1): (-0.75+0j)
Z(0): (-0.75+0j)
Z(0) @ Z(1): (0.25+0j)
Z(2): (-0.75+0j)
Z(0) @ Z(2): (0.25+0j)
Z(3): (-0.75+0j)
Z(0) @ Z(3): (0.25+0j)
Z(1) @ Z(2): (0.25+0j)
Z(1) @ Z(3): (0.25+0j)
Z(2) @ Z(3): (0.25+0j)
